In [44]:
# Imports

import os
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
import matplotlib.image as mpimage
import seaborn as sn
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score

folder = "D:\ProgrammingBigFiles\Kaggle\Dogs vs Cats\\"

In [12]:
# Loading database

images_paths, labels = [], []
for image_id in os.listdir(folder + "train"):
    images_paths, labels = images_paths + [folder + "train/" + image_id], labels + ["dog" in image_id]
train = pd.DataFrame({"image_path": images_paths, "label": labels})
images_paths = []
for image_id in os.listdir(folder + "test"):
    images_paths += [folder + "test/" + image_id]
final = pd.DataFrame({"image_path:": images_paths})

width, height = 150, 150

In [14]:
# Transfer learning model

model = tf.keras.applications.VGG16(weights="imagenet", include_top=False, input_shape=(width, height, 3))

In [30]:
# Getting features

features = []
for line in train.iterrows():
    image = tf.keras.preprocessing.image.load_img(line[1][0], target_size=(width, height))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    feature = model.predict(image)
    features += [feature]
features = np.array(features)
features = features.reshape(features.shape[0], 8192)

In [34]:
# Transforming output

targets = LabelEncoder().fit_transform(labels)
lr = LogisticRegression(solver="lbfgs")
lr.fit(features, targets)

In [37]:
# Predicting

final_features = []
for line in final.iterrows():
    image = tf.keras.preprocessing.image.load_img(line[1][0], target_size=(width, height))
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    feature = model.predict(image)
    final_features += [feature]
final_features = np.array(final_features)
final_features = final_features.reshape(final_features.shape[0], 8192)

y_predict = lr.predict(final_features)

submission = pd.DataFrame({"id": range(1, len(final_features) + 1), "label": y_predict})
submission.to_csv("submission.csv", index=False)